In [1]:
import torch
MODEL_PATH = "../slurm/composer/local-bert-checkpoints/lrcorrect__monarch-mixer-pretrain-786dim-80m-parameters/ep19-ba100000-rank0.pt"
state_dict = torch.load(MODEL_PATH)
state_dict = state_dict["state"]["model"]
state_dict_without_prefix = {k.replace('model.', ''): v for k, v in state_dict.items()}

In [1]:
from omegaconf import OmegaConf as om
from omegaconf import DictConfig
from typing import cast
import sys 
import os
sys.path.append(os.path.abspath(".."))

from main import build_model
import src.create_bert as bert_module
import src.create_model as model_module
from transformers import AutoTokenizer, Trainer, TrainingArguments



yaml_path = "../yamls/pretrain/micro_dna_monarch-mixer-pretrain-786dim-80m-parameters.yaml"

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")


with open(yaml_path) as f:
    cfg = om.load(f)
cfg = cast(DictConfig, cfg)
print(cfg.max_duration)
model = model_module.create_model(cfg.model.get("model_config"))

Could not import FlashFFTConv!


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


200000ba
Using Monarch Mixer for Sequence Mixing: True
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05


In [2]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6)

In [3]:
model.load_state_dict(state_dict_without_prefix)

<All keys matched successfully>

# Testing that model prediction works

In [14]:
from datasets import load_from_disk
from torch.utils.data import DataLoader
from collate import DataCollatorForLanguageModelingSpan

dataset = load_from_disk("../300")
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

dataloader = DataLoader(dataset["train"], collate_fn=data_collator)

In [28]:
seq = next(iter(dataloader))
# we have to unpack the dict
model(**seq)

MaskedLMOutput(loss=tensor(1.5172, grad_fn=<NllLossBackward0>), logits=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
model.push_to_hub("dnam2_100k", safe_serialization=False)

pytorch_model.bin:   0%|          | 0.00/361M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuluHu/dnam2_100k/commit/743b13998e778ffa4b778bf5707f1f8a6e586826', commit_message='Upload BertForMaskedLM', commit_description='', oid='743b13998e778ffa4b778bf5707f1f8a6e586826', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, BertModel
model = BertModel.from_pretrained("togethercomputer/m2-bert-80M-2k", trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type m2_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at togethercomputer/m2-bert-80M-2k and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'e

In [23]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): MonarchMixerSequenceMixing(
            (filter_fn): HyenaFilter(
              (dropout): Dropout(p=0.2, inplace=False)
              (pos_emb): PositionalEmbedding()
              (implicit_filter): Sequential(
                (0): Linear(in_features=5, out_features=128, bias=True)
                (1): Sin()
                (2): Linear(in_features=128, out_features=128, bias=True)
                (3): Sin()
                (4): Linear(in_features=128, out_features=128, bias=True)
                (5): Sin()
                (6)